In [1]:
import pandas as pd

from src import DATA_PATH

In [2]:
# Read some file containing model parameters and filter for a federal state
df_parameters = pd.read_csv(DATA_PATH / 'model_parameters.tsv', sep='\t')
df_parameters = df_parameters[df_parameters.state == 'Bayern']
df_parameters

,state,sim_endpoints,sim_t_step,f_art0,f_art1,f_art2,f_art3,f_art4,f_art5,f_art6,...,k_prep4,k_prep5,k_prep6,n_msm0,n_msm1,n_msm2,n_msm3,n_msm4,n_msm5,n_msm6
1,Bayern,"[972, 1063, 1185, 1276, 1429, 1519, 1826]",1,1345.49168,926.192566,894.371182,853.418207,824.097146,777.052063,750.643063,...,0.000448,-0.000172,0.000309,10744,10744,10744,10744,10744,10744,10744


### Model() class
The `Model()` class is a simple class, receiving a function, i.e. the mathematical model, as well as a list of parameter IDs and function IDs. Make sure, that the IDs are in the same order as they are read by the model.

Instead of defining your own model, you can just import the PrEP Model: `from src.models import prep_model`

However, for demonstration purposes we will build the model from scratch.

In [3]:
from src.models import Model

c_intermitted = 0.189 / 0.58 + 0.811 / 0.91
c_shi = 1 / 0.895

def model(t, y, p):
    y_art, y_prep = y
    k_art, k_prep, n_msm = p
    f_art = k_art * y_art
    f_prep = k_prep * (n_msm - c_intermitted * c_shi * y_prep)
    return f_art, f_prep

f_ids = ['f_art', 'f_prep']
p_ids = ['k_art', 'k_prep', 'n_msm']
prep_model = Model(model, fids=f_ids, pids=p_ids)

The simulator class is capable of running multiple successive simulations, using different parameter and initial values in each simulation. In the end, all simulations are merged into a single time-course.

The class offers two different functions to simulate:
* simulate(): Each simulation uses initial values passed to the function. 
* simulate_continuous(): Only the first simulation uses the initial value passed to the function. All subsequent simulations  use the last value from the previous simulation as their initial value.

Both functions expect the same arguments:
* `endpoints`: List of integers, containing the endpoint of each simulation.\
    *E.g. [100, 150, 300] -> The first simulation runs from t=0 to t=100, the second from t=101 to t=150 and the third from t=151 to t=300*
* `t_step`: Integer. Step size
* `parameters`: 2D array, containing parameter sets for each simulation. Parameters must be in the same order as defined in the model object.
* `y0`: 2D array, containing initial values for each simulation.\
    *Note: `simulate_continuous` only uses the first array

In [4]:
from src.optimization.simulate import Simulator

# initialize Simulator()
sim = Simulator(prep_model)

# simulation endpoints are stored as a string in the dataframe. Convert it to a list
sim_endpoints = [int(i) for i in df_parameters.sim_endpoints.values[0][1:-1].split(',')]   # convert string to list
t_step = df_parameters['sim_t_step'].values[0]

# build parameter and y0 arrays
p = []
y0 = []
for k, _ in enumerate(sim_endpoints):
    p.append([df_parameters[f"{pid}{k}"].values[0] for pid in p_ids])
    y0.append([df_parameters[f"{fid}{k}"].values[0] for fid in f_ids])
    
from pprint import pprint
print(f"endpoints: {sim_endpoints}")
print("\nparameters ([[<k_art, k_prep, n_msm>], ...]):")
pprint(p)
print("\ny0 ([[<f_art, f_prep>], ...]):")
pprint(y0)
    

endpoints: [972, 1063, 1185, 1276, 1429, 1519, 1826]

parameters ([[<k_art, k_prep, n_msm>], ...]):
[[-0.0003841899656764, 0, 10744],
 [-0.0003841899656764, 0.0016008888337117, 10744],
 [-0.0003841899656764, -0.000183613708653, 10744],
 [-0.0003841899656764, 4.6039910324270965e-08, 10744],
 [-0.0003841899656764, 0.0004482630072247, 10744],
 [-0.0003841899656764, -0.0001723313861337, 10744],
 [-0.0003841899656764, 0.0003093489281485, 10744]]

y0 ([[<f_art, f_prep>], ...]):
[[1345.4916798093575, 0.0],
 [926.192566332317, 0.0],
 [894.3711817797828, 1419.909140014262],
 [853.4182071301542, 1219.449681059438],
 [824.0971458947173, 1219.4877467911854],
 [777.0520633470854, 1814.4461992967947],
 [750.6430632366609, 1684.7140622878933]]


In [6]:
sim_results = sim.simulate_continuous(endpoints=sim_endpoints, t_step=t_step, parameters=p, y0=y0)
sim_results.t    # x-values (time)
sim_results.y    # y-values


array([[1345.49167981, 1344.97485469, 1344.4582281 , ...,  667.64214415,
         667.385692  ,  667.12933837],
       [   0.        ,    0.        ,    0.        , ..., 2433.22453935,
        2435.52411616, 2437.82272583]])